In [1]:
import imports
import importlib
importlib.reload(imports)
from imports import *

# https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html
# http://opencorpora.org/dict.php?act=gram
# https://github.com/pymorphy2/pymorphy2/blob/92d546f042ff14601376d3646242908d5ab786c1/pymorphy2/tagset.py#L130
feature_tags_array = [
    OpencorporaTag.PARTS_OF_SPEECH, # часть речи
    OpencorporaTag.GENDERS, # род
    OpencorporaTag.NUMBERS, # число
    OpencorporaTag.CASES, # падеж
    OpencorporaTag.ASPECTS, # соверш / несоверш
    OpencorporaTag.TRANSITIVITY, # перех / непереходный
    OpencorporaTag.PERSONS, # лицо
    OpencorporaTag.TENSES, # время
    OpencorporaTag.MOODS, # наклонение
    OpencorporaTag.VOICES, # залог
    #INVOLVEMENT
    ['Prnt'], # вводные слова
    ['Apro'], # местоимение
    ['NUMB'], # число вида 1234
    ['LATIN'], # текст на английском
    ['UNKN'], # неизвестный токен
    ['PUNCT_DASH', 'PUNCT_DOT', 'PUNCT_COMMA', 'PUNCT_QUOTE',
     'PUNCT_LEFT_PARENTHESIS', 'PUNCT_RIGHT_PARENTHESIS' ], # "()
    ['CAPITALIZED'], # начинается с заглавной буквы
    ['Fixd', 'Abbr'] # неизменяемое, сокращение
]

CUT_NAVEC_TAGS_ARRAY = [
    #'NOUN', #'ADJF'
]

params = build_params({
    "VARIANTS_CNT": 3,
    "TARGET_CLASSES_COUNT": 3,
    "INPUT_WORDS_CNT": 32,
    "feature_tags_array": feature_tags_array,
    "PUNCTUATION_TARGET": {
        "$empty": NO_PUNCT,
        ",": 1,
        ".": 2,
        "!": 2,
        "?": 2,
    },
    "USE_NAVEC": True,
    'CUT_NAVEC_TAGS_SET': set(CUT_NAVEC_TAGS_ARRAY),
    "RETAIN_LEFT_PUNCT": True,
    'type': 'lenta',
    'NON_PUNCT_PROB': 0.5,

    'INFECTED_TEXT_PROB': 0.5,
    'INFECT_TYPE_PROBS': {
        'REPLACE_UNDEF': 0.1,
        'INCORRECT_PUNCT_RANDOM_PLACE': 0.1,
        'INCORRECT_PUNCT_CENTER': 0.2,
        'CORRECT_PUNCT_CENTER': 0.3,
        'CORRECT_PUNCT_RIGHT': 0.3,
    }
})
# print("NON_PUNCT_PROB': 0.3,\n" * 10)
# print("INFECTED_TEXT_PROB': 0\n" * 10)


In [ ]:
Брать 10% каждого чанка и при запуске проверять до / после и смотреть, было ли улучшение

In [3]:
random.seed(42)
np.random.seed(42)

import dataset_builder
importlib.reload(dataset_builder)

dataset_builder.create_dataset_for_text("Привет, тест. это тест. Тест.", params)[2:4]


([['PAD', 'PAD', 'PAD', 'PAD', ' #$empty# ', '.', 'Привет', 'тест', 'это'],
  ['PAD', 'PAD', 'PAD', 'Привет', ' #,# ', '.', 'тест', 'это', 'тест'],
  ['PAD', 'Привет', ',', 'тест', ' #.# ', 'это', 'тест', 'Тест', 'PAD'],
  [',', 'тест', 'UNDEF', 'это', ' #$empty# ', 'тест', 'Тест', 'PAD', 'PAD'],
  ['тест', '.', 'это', 'тест', ' #.# ', 'Тест', 'PAD', 'PAD', 'PAD'],
  ['это', 'тест', '.', 'Тест', ' #.# ', 'PAD', 'PAD', 'PAD', 'PAD']],
 tensor([3, 3, 0, 1, 0, 0], dtype=torch.uint8))

In [ ]:

server = run_server_if_not_running()
server_install_packages(server)

# server.rpc_simple(dataset_builder.get_word_features, 'кошка', params).shape
# server.rpc_simple(dataset_builder.create_dataset, ['а, б'], params, False)[0].shape

In [2]:
params["train_test_split"] = 0.9
# lenta -- 50

params["chunk_size"] = 10 # 3000 # 300000
params["batch_size"] = 20000
params["max_parallel"] = 3
params["type"] = "lenta"

params["max_last_read_queue_size"] = 1

class AutoWriter:
    def __init__(self, name, storage) -> None:
        self.name = name
        self.storage = storage
        self.x = None

    def write_splitted(self, x, cnt, cnt_per_elem):
        if self.x == None:
            self.x = x
        elif isinstance(x, torch.Tensor):
            self.x = torch.cat((self.x, x))
        else:
            assert isinstance(x, list)
            self.x += x
        
        llen = self.x.shape[0] if isinstance(self.x, torch.Tensor) else len(self.x)
        for i in range(0, llen, cnt_per_elem):
            if i + cnt_per_elem < llen:
                vv = self.x[i: i + cnt_per_elem]
                self.storage.store(self.name, cnt, 
                                   vv.clone() if isinstance(self.x, torch.Tensor)
                                   else vv)
                cnt += 1
            else:
                self.x = self.x[i:]
                break

        return cnt
        
    def write(self, x, cnt):
        cnt_per_elem = math.floor(x.shape[0] / 
                                    size_of_tensor(x) * 
                                    (50 * 1024 * 1024))
        cnt = self.write_splitted(x, cnt, cnt_per_elem)
        return cnt_per_elem, cnt

class AsyncDatasetWriter:
    def __init__(self, rpc_server):
        self.storage = Storage("cache2/storage2")
        self.storage.clear()
        self.chunks_count = 0

        self.storage.write_meta("chunks_count", 0)
        self.storage.write_meta("params", params)

        self.parallel_count = threading.Semaphore(params["max_parallel"])
        self.write_queue = queue.Queue()
        # self.thread_write = threading.Thread(target=asyncio.run, args=(self.write_task(),))
        
        self.device = torch.device('cuda:0')

        self.chunks_iter = None
        self.rpc_server = rpc_server

        self.x_a = AutoWriter("x", self.storage)
        self.y_a = AutoWriter("y", self.storage)
        self.text_res_a = AutoWriter("text_res", self.storage)
        self.is_infected_a = AutoWriter("is_infected", self.storage)
        
        self.thread_write = threading.Thread(target=self.write_task)
        self.thread_write.start()
        
        
    def chunk_loaded_callback(self, future):
        print("chunk_loaded_callback")
        self.write_queue.put(future)
  

    def test_loaded_callback(self, future):
        print("got test")
        x, y, text_res, is_infected = future.get_result()
        print("writing test started cnt=", x.shape[0], "size=", size_of_tensor(x) / 1024 / 1024, "MB")
        self.storage.write_meta("x_test", x)
        self.storage.write_meta("y_test", y)
        self.storage.write_meta("text_res_test", text_res)
        self.storage.write_meta("is_infected_test", is_infected)
        print("writing test finsihed")

    def has_free_space(self):
        free_space_gb = shutil.disk_usage(self.storage.path).free / 1024 / 1024 / 1024
        if free_space_gb < 5:
            print(f"Out of disk space. Remainging {free_space_gb} GB")
            return False
        return True
        



    def write(self, future, i):
        if not self.has_free_space(): return
        # if future.exception() is not None:
        #     future.
            # absprint("ERROR: ", future.exception(), "\n", 
                    # "\n".join(traceback.format_tb(future.traceback())  ))
            # raise future.exception()
        x, y, text_res, is_infected = future.get_result()
        print("writing started", size_of_tensor(x) / 1024 / 1024, "MB")
        cnt_per_elem, new_i = self.x_a.write(x, i)
        assert self.y_a.write_splitted(y, i, cnt_per_elem) == new_i
        assert self.text_res_a.write_splitted(text_res, i, cnt_per_elem) == new_i
        assert self.is_infected_a.write_splitted(is_infected, i, cnt_per_elem) == new_i
        self.parallel_count.release()
        self.write_queue.task_done()
        print("writing finished")
        self.chunks_count += (new_i - i)
        print("chunks_count", self.chunks_count)
        self.storage.write_meta("chunks_count", self.chunks_count)
        return new_i

    def write_task(self):        
        try:
            print("write_task started")
            keep_running = True        
            def handle_executor_done_callback(future):
                nonlocal keep_running
                try:
                    future.result()
                except Exception as e:
                    print("ERROR", "writer thread failed:\n", type(e).__name__, e)
                    print("\n".join(traceback.format_tb(e.__traceback__)))
                    keep_running = False

            chunk_number = 0
            while keep_running:
                chunk_number = self.write(self.write_queue.get(), chunk_number)

            # with concurrent.futures.ThreadPoolExecutor(params["max_parallel"], "WRITER") as executor:
            #     chunk_number = 0
            #     while keep_running:
            #         if not self.has_free_space(): return
            #         future = executor.submit(self.write, self.write_queue.get(), chunk_number)
            #         future.add_done_callback(handle_executor_done_callback)
            #         chunk_number += 1

        except Exception as e:
            print("ERROR", "writer thread failed:\n", type(e).__name__, e)
            print("\n".join(traceback.format_tb(e.__traceback__)))
    
    def load_dataset(self, stream):
        # lenta -- 200
        test_chunk = stream.get(50)
        future = self.rpc_server.rpc_async(dataset_builder.create_dataset, test_chunk, params, False)
        future.set_callback(self.test_loaded_callback)
        
        # with concurrent.futures.ThreadPoolExecutor(max_workers=params["max_parallel"]) as executor:
        for chunk in stream.group(params["chunk_size"]):
            if not self.thread_write.is_alive(): return
            self.parallel_count.acquire()

            print("submit task")
            future = self.rpc_server.rpc_async(dataset_builder.create_dataset, chunk, params, False)
            future.set_callback(self.chunk_loaded_callback)
            future = None
        
        
from dataset_lib import get_lenta_records, get_writers_records_stream
server = run_server_if_not_running()
server_install_packages(server)
# stream = Stream(get_lenta_records()).skip(5000).limit(200_000).map(lambda record: record.text)
stream = get_writers_records_stream().limit(200_000)
writer = AsyncDatasetWriter(server)
writer.load_dataset(stream)
print("DONE")

Server is listening on 0.0.0.0:65231
Connected by ('51.250.1.213', 38036)
write_task started
submit task
submit task
submit task
got test
writing test started cnt= 13274 size= 396.1783447265625 MB
writing test finsihed
chunk_loaded_callback
getting result started
BasicAsyncResult completed with result `(tensor([[[ 0.0000, ...`
1163 1163
writing started 34.71112060546875 MB
writing finished
chunks_count 0
submit task
chunk_loaded_callback
getting result started
BasicAsyncResult completed with result `(tensor([[[ 0.0000, ...`
1689 1689
writing started 50.41021728515625 MB
writing finished
chunks_count 1
submit task
chunk_loaded_callback
getting result started
BasicAsyncResult completed with result `(tensor([[[ 0.0000, ...`
4429 4429
writing started 132.18878173828125 MB
writing finishedsubmit task

chunks_count 4
chunk_loaded_callback
getting result started
BasicAsyncResult completed with result `(tensor([[[ 0.0000, ...`
4542 4542
writing started 135.5614013671875 MB
writing finishedsubm

In [19]:
class Kek:
    def __init__(self, a):
        self.a = a
    
    def read(self, cnt=-1): 
        print(cnt)
        return self.a.read(cnt)
    def readline(self): raise Exception("no realization")
    def readable(self): return True

    def write(self, buf):
        print(len(buf))

import io
import dill

a = Kek(io.BytesIO())

dill.dump(["kek"] * 1000, a)

# Reset the position of the BytesIO object to the beginning
a.a.seek(0)

# Pass the BytesIO object directly to the Kek class
a.a.read()

2
9
2009


b''